# Netmiko告警邮件实验
* smtplib 用于发送邮件
* email 用于创建邮件
* netmiko 用于连接设备监控配置
* time 用于设定监控间隔
* 说明企业邮箱问题

In [1]:
import time
import smtplib
from email.header import Header
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from netmiko import ConnectHandler

## 创建邮件与smtp对象

In [2]:
class MAIL_SENDER():
    def __init__(self, usermail, password, obj_address, smtpserver, smtpport=25):
        self.usermail = usermail
        self.password = password
        self.obj_address = obj_address
        self.smtpserver = smtpserver
        self.smtpport = smtpport
        
    def create_mail(self, subject, text):
        msg = MIMEMultipart('mixed')
        msg['From'] = self.usermail
        msg['To'] = self.obj_address
        msg['Subject'] = Header(subject, 'utf-8')
        text_sub = MIMEText(text, 'plain', 'utf-8')
        msg.attach(text_sub)
        return msg.as_string()
        
    def send(self, msg):
        try:
            sender = smtplib.SMTP(self.smtpserver, self.smtpport)
            sender.login(self.usermail, self.password)
            sender.sendmail(self.usermail, self.obj_address, msg)
            print("检测到故障，已创建故障邮件，发送成功！")
            sender.quit()
            
        except Exception as e:
            print("发送失败，错误如下：")
            print(e)

## 设置设备登录信息

In [3]:
device_info = {
    "device_type": "cisco_ios",
    "ip" : ["192.168.1.60","192.168.1.61", "192.168.1.62"], 
    "port" : "22", 
    "username" : "ciscouser",
    "password" : "cisco@123",
    "secret" : "cisco123",
    "verbose": True
}

## 完成对接口状态的监控
* 循环监控接口状态
* 当接口出现DOWN的状态时发送邮件告警

In [ ]:
def check_IF_state(device_info):
    reporter = MAIL_SENDER("ccxxll23@tom.com", "sdfsdf324", "154354595@qq.com", "smtp.tom.com")
    # 连接所有设备
    server_pool = []
    for ip in device_info["ip"]:
        device = device_info.copy()
        device["ip"] = ip
        server_pool.append(ConnectHandler(**device))
        
    # 无限循环运维
    while True:
        results = []
        for server in server_pool:
            info = server.send_command("show ip interface brief")
            results.append(info)
        report_err(device_info, reporter, results)
        time.sleep(30)
    #server.disconnect()

def report_err(device_info, reporter, results):
    for index in range(len(results)):
        this_ip = device_info["ip"][index]
        device = results[index].split("\n")[1:]# 去除第一行无用信息
        for interface in device:
            if "down" in interface:
                if_name = interface.split()[0]
                msg = reporter.create_mail("告警邮件", "设备 {} 的接口 {} 出现故障，请尽快修复！".format(this_ip, if_name))
                reporter.send(msg)

check_IF_state(device_info)

SSH connection established to 192.168.1.60:22
Interactive SSH session established
SSH connection established to 192.168.1.61:22
Interactive SSH session established
SSH connection established to 192.168.1.62:22
Interactive SSH session established
检测到故障，已创建故障邮件，发送成功！
